# Data Preprocessing


Before using interpolation methods and outlier removal, I'll apply certain changes to only get the relevant data and give it a certain format. These changes will mostly be: 
- Editing column names 
- Eliminating irrelevant features to the project
- Accurately replacing null values
- Determine column data types
- Extract relevant instances 
- Reduce number of instances


In [2]:
import pandas as pd
import numpy as np

## Edit column names and eliminate innecessary features

The data was obtained from the official page of the SEMADET link here, describe columns 

In [57]:
filepath = "datasets/semadet-aire-2022.csv"

df = pd.read_csv(filepath,
                     encoding='utf-8',
                     parse_dates=[1],
                     )

df.columns = df.columns.str.lower().str.strip()

df.rename({"pm2.5": "pm25", 
           "date_time": "date",
           "precipitacion": "pp", 
           "rad solar": "rs",
           "presion barometrica": "pba"},
          axis="columns",
          inplace=True)

df.drop(["rs", "uvi", "nox", "no", "tmpi"], axis="columns", inplace=True)

/var/folders/b6/r8x8vgk95n1gjf60w26y4dr80000gn/T/ipykernel_11378/2730506958.py:3: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath,


In [31]:
df.head()

,estacion,date,hora,o3,no2,so2,co,pm10,pm25,tmp,rh,ws,wd,pp,pba
0,Oblatos,1/1/2022,12:00:00 AM,IF,IF,IF,IF,IF,IF,IF,IR,IF,IF,IF,NaN
1,Oblatos,1/1/2022,1:00:00 AM,IF,IF,IF,IF,IF,IF,IF,IR,IF,IF,IF,NaN
2,Oblatos,1/1/2022,2:00:00 AM,IF,IF,IF,IF,IF,IF,IF,IR,IF,IF,IF,NaN
3,Oblatos,1/1/2022,3:00:00 AM,IF,IF,IF,IF,IF,IF,IF,IR,IF,IF,IF,NaN
4,Oblatos,1/1/2022,4:00:00 AM,IF,IF,IF,IF,IF,IF,IF,IR,IF,IF,IF,NaN


## Replace null values

I will replace all the identifiers for the null values (which are quite varied) with nan to see which columns to keep based on amount of values.

In [58]:
def replace_with_null(row):
    null_values = ["IO", "SE", "ND", "IF", "VE", "IR", "VZ", "IC", "IR 1000", "IR valor 1000"]
    exclude_columns = ["estacion", "date", "hora"]
    
    for column in row.index:
        if column not in exclude_columns and row[column] in null_values:
            row[column] = np.nan
            
    return row
    

In [59]:
df = df.apply(replace_with_null, axis="columns")

Sice the no2 and so2 columns have no useful data, they'll be dropped entirely.

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87601 entries, 0 to 87600
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   estacion  87601 non-null  object 
 1   date      87601 non-null  object 
 2   hora      87601 non-null  object 
 3   o3        52589 non-null  object 
 4   no2       0 non-null      float64
 5   so2       12182 non-null  object 
 6   co        17050 non-null  object 
 7   pm10      47037 non-null  object 
 8   pm25      22705 non-null  object 
 9   tmp       28929 non-null  object 
 10  rh        47008 non-null  object 
 11  ws        50641 non-null  object 
 12  wd        17188 non-null  object 
 13  pp        52454 non-null  object 
 14  pba       38510 non-null  object 
dtypes: float64(1), object(14)
memory usage: 10.0+ MB


In [60]:
df.drop(["no2", "so2"], axis="columns", inplace=True)

## Specify data tyoes

Now that the null values have been replaced, I can specify the correct data type for each column.

In [61]:
float_cols = ["o3", "co", "pm10", "pm25", "tmp", "rh", "ws", "wd", "pp", "pba"]
df[float_cols] = df[float_cols].astype('float')

## Extract relevant instances

The AQI daily forecasting will only be done for the Tlaquepaque region, so I'll also extract all the pertaining rows and can also now drop the column estacion. I'll also convert the date column into the index.

In [62]:
df = df[df["estacion"] == "Tlaquepaque"]

In [63]:
df.drop("estacion", axis="columns", inplace=True)

In [64]:
df.index = pd.to_datetime(df['date'], format='%d/%m/%Y')
df.drop("date", axis="columns", inplace=True)

In [65]:
df= df.sort_index()

In [66]:
df.head()

,hora,o3,co,pm10,pm25,tmp,rh,ws,wd,pp,pba
date,,,,,,,,,,,
2022-01-01,12:00:00 AM,0.02,NaN,59.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01,1:00:00 AM,0.01,NaN,49.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01,2:00:00 AM,0.00,NaN,59.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01,3:00:00 AM,0.01,NaN,50.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01,4:00:00 AM,0.01,NaN,35.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Reduce number of instances

Finally, since I want to forecast the daily AQI, I'll only leave one entry for each day. It'll be the average of every feauture, except wind direction, which should use the cirular mean, which is a way to calculate the average of angular data.

In [67]:
# Define a function for circular mean
def circular_mean(angles):
    angles_rad = np.deg2rad(angles)  # Convert degrees to radians
    mean_sin = np.mean(np.sin(angles_rad))
    mean_cos = np.mean(np.cos(angles_rad))
    mean_angle = np.arctan2(mean_sin, mean_cos)  # Compute mean angle
    return np.rad2deg(mean_angle) % 360  # Convert back to degrees and normalize

In [68]:
aggregation_functions = {col: "mean" for col in df.columns if col not in ["wd", "hora"]}
aggregation_functions["wd"] = circular_mean

In [69]:
df_daily = df.groupby("date").agg(aggregation_functions)

After grouping the values by dates, the pba (Barometric Pressure) and co (Carbon dioxide) has no relevant information at all, so I'll eliminate it.

In [70]:
df_daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 365 entries, 2022-01-01 to 2022-12-31
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   o3      232 non-null    float64
 1   co      0 non-null      float64
 2   pm10    168 non-null    float64
 3   pm25    17 non-null     float64
 4   tmp     66 non-null     float64
 5   rh      258 non-null    float64
 6   ws      258 non-null    float64
 7   pp      285 non-null    float64
 8   pba     0 non-null      float64
 9   wd      258 non-null    float64
dtypes: float64(10)
memory usage: 31.4 KB


In [71]:
df_daily.drop(["pba", "co"], axis="columns", inplace=True)

In [72]:
df_daily.head()

,o3,pm10,pm25,tmp,rh,ws,pp,wd
date,,,,,,,,
2022-01-01,0.022083,58.065000,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-02,0.016957,52.771304,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-03,0.024167,75.727917,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,0.016667,111.918000,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-05,0.017500,75.196250,NaN,NaN,NaN,NaN,NaN,NaN


I'll just corroborate that all the months have complete dates.

In [74]:
year = df_daily.index.year
month = df_daily.index.month
dates_per_month = df_daily.groupby([year, month]).size().unstack(fill_value=0)
dates_per_month.index.name = 'year'
dates_per_month.columns.name = 'month'
dates_per_month

month,1,2,3,4,5,6,7,8,9,10,11,12
year,,,,,,,,,,,,
2022,31,28,31,30,31,30,31,31,30,31,30,31


## Save pre processed data

Now that the data has been properly processed, I'll save it in a new file.

In [53]:
filename = "semadet-tlaquepaque-2022"
df_daily.to_csv(f"datasets/{filename}.csv")